In [22]:
import pandas as pd
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, lpSum

In [23]:
cricket_df = pd.read_csv("rcb2022.csv")
# cricket_df

In [24]:
def clean_data(df_features):

    df = df_features.copy()

    df['quantity'] = 1

    df['selection_popularity'] = df['SelectionPercent'].apply(lambda x:int(x[:-1:]))
    df['selection_popularity'] = (df['selection_popularity'] * df['Price(IndCrores)'])/100

    df = pd.get_dummies(df, columns=['Leader', 'PlayerRole', 'Nationality'])

    PlayerName = list(df['PlayerName'])
    player_dict = {}
    for col in df.columns:
        player_dict[col] = dict(zip(PlayerName, df[col].values))   
    return PlayerName, player_dict     

In [25]:
def team_optimization(player_name, features: dict): ##MIGHT CHANGE

    player_chosen = LpVariable.dict("Player Chosen", player_name, 0, 1, cat=int) #####MIGHT CHANGE

    # Title of LP
    prob = LpProblem("Optimize_Playing11", LpMaximize)

    # Objective Function
    prob += lpSum([features['selection_popularity'][i] * player_chosen[i]for i in player_name]), "Maximize Points based on Selection Popularity"

    # Total Selection 
    prob += lpSum([features['quantity'][f] * player_chosen[f] for f in player_name]) == 11, "Playing 11"

    # Captain 
    prob += lpSum([features['Leader_Captain'][f] * player_chosen[f] for f in player_name]) == 1, "Captain"

    # Vice Captain 
    prob += lpSum([features['Leader_ViceCaptain'][f] * player_chosen[f] for f in player_name]) == 1, "Vice Captain"

    # Max and Min AllRounder
    prob += lpSum([features['PlayerRole_AllRounder'][f] * player_chosen[f] for f in player_name]) <= 3, "Max AllRounder"
    prob += lpSum([features['PlayerRole_AllRounder'][f] * player_chosen[f] for f in player_name]) >= 2, "Min AllRounder"

    # Max and Min Batsmen
    prob += lpSum([features['PlayerRole_Batsmen'][f] * player_chosen[f] for f in player_name]) <= 5, "Max Batsmen"
    prob += lpSum([features['PlayerRole_Batsmen'][f] * player_chosen[f] for f in player_name]) >= 3, "Min Batsmen"

    # Max and Min Bowler 
    prob += lpSum([features['PlayerRole_Bowler'][f] * player_chosen[f] for f in player_name]) <= 5, "Max Bowler"
    prob += lpSum([features['PlayerRole_Bowler'][f] * player_chosen[f] for f in player_name]) >= 3, "Min Bowler"

    # Wicketkeeper 
    prob += lpSum([features['PlayerRole_WicketKeeper'][f] * player_chosen[f] for f in player_name]) >= 1, "Min WicketKeeper"

    # Indian Players 
    prob += lpSum([features['Nationality_Indian'][f] * player_chosen[f] for f in player_name]) <= 10, "Max Indian Players"
    prob += lpSum([features['Nationality_Indian'][f] * player_chosen[f] for f in player_name]) >= 7, "Min Indian Players"

    # Foreign Players 
    prob += lpSum([features['Nationality_Overseas'][f] * player_chosen[f] for f in player_name]) <= 4, "Max Overseas Players"
    prob += lpSum([features['Nationality_Overseas'][f] * player_chosen[f] for f in player_name]) >= 1, "Min Overseas Players"

    LpFile = prob.writeLP("Playing11_Cricket")

    prob.solve()
    print("Status = ", LpStatus[prob.status])
    # print(prob.objective)
    # print(prob.constraints)

    return prob

In [26]:
player_names, feature = clean_data(cricket_df)
prob = team_optimization(player_names, feature)

Status =  Optimal


In [27]:
player_roles = dict(zip(cricket_df['PlayerName'], cricket_df['PlayerRole']))

players_choosen = []
for player in prob.variables():
    if player.varValue > 0:
        act_name = " ".join(player.name.split('_')[-2:])
        players_choosen.append(act_name)
        print(player.name, ' = ', player_roles[act_name])


print(players_choosen)
print("Total Players chosen = {}".format(len(players_choosen)))

Player_Chosen_Aneeshwar_Gautam  =  Batsmen
Player_Chosen_Anuj_Rawat  =  WicketKeeper
Player_Chosen_Dinesh_Karthik  =  WicketKeeper
Player_Chosen_Faf_DuPlessis  =  Batsmen
Player_Chosen_Glenn_Maxwell  =  AllRounder
Player_Chosen_Harshal_Patel  =  Bowler
Player_Chosen_Josh_Hazlewood  =  Bowler
Player_Chosen_Mohammed_Siraj  =  Bowler
Player_Chosen_Shahbaz_Ahmed  =  AllRounder
Player_Chosen_Virat_Kohli  =  Batsmen
Player_Chosen_Wanindu_Hasaranga  =  AllRounder
['Aneeshwar Gautam', 'Anuj Rawat', 'Dinesh Karthik', 'Faf DuPlessis', 'Glenn Maxwell', 'Harshal Patel', 'Josh Hazlewood', 'Mohammed Siraj', 'Shahbaz Ahmed', 'Virat Kohli', 'Wanindu Hasaranga']
Total Players chosen = 11


In [30]:
result_df = cricket_df[cricket_df['PlayerName'].isin(players_choosen)]
result_df.sort_values('Leader')
result_df.to_csv('RCBPlayingXI.csv', index=None)


In [31]:
result_df.sort_values('Leader')

,PlayerName,Price(IndCrores),SelectionPercent,Leader,PlayerRole,Nationality
3,Faf DuPlessis,7.00,91%,Captain,Batsmen,Overseas
0,Virat Kohli,15.00,93%,ViceCaptain,Batsmen,Indian
1,Glenn Maxwell,11.00,87%,NaN,AllRounder,Overseas
2,Mohammed Siraj,7.00,89%,NaN,Bowler,Indian
4,Harshal Patel,10.75,88%,NaN,Bowler,Indian
5,Wanindu Hasaranga,10.75,84%,NaN,AllRounder,Overseas
6,Dinesh Karthik,5.50,85%,NaN,WicketKeeper,Indian
7,Josh Hazlewood,7.75,86%,NaN,Bowler,Overseas
8,Shahbaz Ahmed,2.40,86%,NaN,AllRounder,Indian
9,Anuj Rawat,3.40,72%,NaN,WicketKeeper,Indian
